In [96]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
sentences = open('data/tinyshakespeare.txt').read()

In [12]:
chars = sorted(set(sentences))

In [21]:
itoc = {i:c for i,c in enumerate(chars)}
ctoi = {c:i for i,c in itoc.items()}

In [23]:
len(sentences)

1115394

In [24]:
len(chars)

65

In [25]:
N = torch.zeros((65,65), dtype=torch.int32)

In [26]:
for i in range(len(sentences) - 1):
    ix1 = ctoi[sentences[i]]
    ix2 = ctoi[sentences[i+1]]
    N[ix1,ix2] += 1

In [31]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)

In [52]:
for i in range(1):
    out = []
    ix = 0
    for _ in range(50):
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itoc[ix])
    print(''.join(out))


A oe d Homano le.
Catare, nge.
TES loteth te anto


Neural Network based approach

In [53]:
# build dataset
xs, ys = [], []
for ch1, ch2 in zip(sentences, sentences[1:]):
    ix1 = ctoi[ch1]
    ix2 = ctoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [56]:
xs.shape, ys.shape

(torch.Size([1115393]), torch.Size([1115393]))

In [60]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [59]:
class MLP(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(len(chars), hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, len(chars))
        )

    def forward(self, x):
        return self.net(x)

In [61]:
model = MLP(100)

In [76]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [78]:
for n in range(10000):
    batch_indices = torch.randint(0, xs.shape[0] - 1, (64,))
    inp = F.one_hot(xs[batch_indices], num_classes=65).float()
    tgt = F.one_hot(ys[batch_indices], num_classes=65).float()
    logits = model(inp)
    loss = F.cross_entropy(logits, tgt)
    print(loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

2.5261874198913574
2.5404841899871826
2.5172088146209717
2.612074375152588
2.805495500564575
2.4772331714630127
2.706348419189453
2.500718116760254
2.3783371448516846
2.594080924987793
2.543304681777954
2.864797592163086
2.606098175048828
2.274022340774536
2.8629941940307617
2.6015026569366455
2.5985159873962402
2.762082576751709
2.6412010192871094
2.7464280128479004
2.3586692810058594
2.645843267440796
2.93221116065979
2.517393112182617
2.7330098152160645
2.710319757461548
2.5635881423950195
2.5303072929382324
2.7365384101867676
2.5466346740722656
2.4970648288726807
2.5093793869018555
2.8105170726776123
2.5313799381256104
2.998734474182129
2.5314764976501465
2.2460038661956787
2.6294212341308594
2.8307971954345703
2.608422040939331
2.537247896194458
2.575484275817871
2.5926566123962402
2.843923568725586
2.8222146034240723
2.660691022872925
2.4242076873779297
2.69411301612854
2.7325563430786133
2.620504856109619
2.4372262954711914
2.4485411643981934
2.726430892944336
2.736124038696289


In [95]:
for i in range(1):
    out = []
    ix = torch.randint(0, len(chars), (1,))

    for _ in range(50):
        xenc = F.one_hot(torch.tensor([ix]), num_classes=65).float()
        logits = model(xenc)
        probs = F.softmax(logits, dim=-1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item()
        out.append(itoc[ix])
    print(''.join(out))

GenceeshecirounE:
To;
For f, t!
LN-udyIWha arWIKoP
